In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# == Download the repo content and install dependencies ==
import shutil
!git clone https://github.com/ifzhang/ByteTrack.git
%cd /content/ByteTrack/
%mkdir pretrained
%cd pretrained

shutil.copy2('/content/drive/MyDrive/billiards_MOT/best_ckpt.pth.tar','./YOLOX_best_ckpt.pth.tar')
# == Download pretrained X model weights ==
#yolox 삭제
#shutil.rmtree('/content/ByteTrack/yolox')

Cloning into 'ByteTrack'...
remote: Enumerating objects: 1921, done.
remote: Total 1921 (delta 0), reused 0 (delta 0), pack-reused 1921
Receiving objects: 100% (1921/1921), 78.16 MiB | 17.31 MiB/s, done.
Resolving deltas: 100% (1091/1091), done.
/content/ByteTrack
mkdir: cannot create directory ‘pretrained’: File exists
/content/ByteTrack/pretrained


'./YOLOX_best_ckpt.pth.tar'

In [4]:
# == Install dependencies ==
!pip3 install cython
!pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip3 install cython_bbox

%cd /content/ByteTrack/
!pip3 install -r requirements.txt

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-v9weq0nb
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-v9weq0nb
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=264361 sha256=5de42925ce99a38d8c185a2352b8c7f0b384627a3d335879023c6d46abe9a4aa
  Stored in directory: /tmp/pip-ephem-wheel-cache-1k3m5aqv/wheels/e2/6b/1d/344ac773c7495ea0b85eb228bc66daec7400a143a92d36b7b1
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.4
    Uninstalling pycocotools-2.0.4:
      Successfully uninstalled pycocotools-2.0.4
     |████████████████████████████████| 41 kB 218 kB/s 
  Created wheel for cython-bbox: filename=cython_bbox-0.1.3-cp37-cp37m-linux_x86_64.whl size=57510 sha256=480a0d778049b1ccdb6c85aeaf48cdabb00178478625cd603b790225c6c71fcb
  Stored in directory: /root/.cache/pip/wheels/51/82/21/5def8bc98ae4ea436d7f0decb7194d20

In [5]:
# == Install ByteTrack ==
!python3 setup.py develop

running develop
running egg_info
creating yolox.egg-info
writing yolox.egg-info/PKG-INFO
writing dependency_links to yolox.egg-info/dependency_links.txt
writing top-level names to yolox.egg-info/top_level.txt
writing manifest file 'yolox.egg-info/SOURCES.txt'
package init file 'pretrained/__init__.py' not found (or not a regular file)
package init file 'deploy/__init__.py' not found (or not a regular file)
package init file 'tutorials/__init__.py' not found (or not a regular file)
package init file 'datasets/__init__.py' not found (or not a regular file)
package init file 'assets/__init__.py' not found (or not a regular file)
package init file 'tools/__init__.py' not found (or not a regular file)
package init file 'videos/__init__.py' not found (or not a regular file)
package init file 'exps/__init__.py' not found (or not a regular file)
package init file 'pretrained/ByteTrack/__init__.py' not found (or not a regular file)
package init file 'pretrained/ByteTrack/deploy/__init__.py' not

In [6]:
!git clone https://github.com/roboflow-ai/YOLOX.git

#%cd YOLOX
#!pip3 install -U pip && pip3 install -r requirements.txt
#!pip3 install -v -e .  

#!pip uninstall -y torch torchvision torchaudio
# May need to change in the future if Colab no longer uses CUDA 11.0
#!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

Cloning into 'YOLOX'...
remote: Enumerating objects: 786, done.
remote: Total 786 (delta 0), reused 0 (delta 0), pack-reused 786
Receiving objects: 100% (786/786), 5.78 MiB | 7.25 MiB/s, done.
Resolving deltas: 100% (416/416), done.


In [7]:
#cfg 파일(classes)이랑 model:yolox_voc_s.py 바꾸기
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [8]:
##REPLACE this cell with your classnames stripped of whitespace and lowercase
%%writetemplate /content/ByteTrack/YOLOX/yolox/data/datasets/voc_classes.py

VOC_CLASSES = (
  "red ball",
  "white ball",
  "yellow ball"
)

In [9]:
NUM_CLASSES = 3
!sed -i -e 's/self.num_classes = 20/self.num_classes = {NUM_CLASSES}/g' "/content/ByteTrack/YOLOX/exps/example/yolox_voc/yolox_voc_s.py"

In [ ]:
#폴더 복사

#shutil.copytree("/content/ByteTrack/YOLOX/yolox", "/content/ByteTrack/yolox")

In [10]:
# run inference demo (can be slow on colab). The cell output is deflected to the file 'log.txt' for downstream use to display the result but feel free to remove 
#./tools/convert_video.py
#./tools/demo_track.py
%cd /content/ByteTrack
!python3 tools/demo_track.py video -f YOLOX/exps/example/yolox_voc/yolox_voc_s.py -c pretrained/YOLOX_best_ckpt.pth.tar --path /content/drive/MyDrive/billiards_MOT/billiards.mp4 --fp16 --fuse --save_result --device gpu &> log.txt

/content/ByteTrack


In [11]:
# == Get rendered result video file path ==
import re
%cd /content/ByteTrack
with open('log.txt', 'r') as file:
    text = file.read().replace('\n', '')

m = re.search('video save_path is ./(.+?).mp4', text)
if m:
    found = '/content/ByteTrack/' + m.group(1) + ".mp4"

found

/content/ByteTrack


'/content/ByteTrack/YOLOX_outputs/yolox_voc_s/track_vis/2022_03_25_11_18_24/billiards.mp4'

In [ ]:
# == Convert mp4 to webm video file for proper display on colab ==
%cd /content/ByteTrack
!ffmpeg -i $found -vcodec vp9 out.webm

/content/ByteTrack
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-l

In [ ]:
# == Display results ==
from IPython.display import HTML
from base64 import b64encode

video_path = '/content/ByteTrack/out.webm'

mp4 = open(video_path, "rb").read()
data_url = "data:video/webm;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=600 controls>
      <source src="{data_url}" type="video/webm">
</video>
""")